In [1]:
#Methods

from typing import Final
from telegram import Update
from telegram.constants import ParseMode
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

import codeforces_api
import random

import os
import json

#Commands
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text('Callate, dime tu user de codeforces')
    

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    #await update.message.reply_text('Generic help message')
    await update.message.reply_text(text="<a href='codeforces.com'>Codeforces</a>", parse_mode=ParseMode.HTML)

async def random_problems_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    problem_links = get_random_problems()
    for link in problem_links:
        await update.message.reply_text(link)


#Handle Responses
def handle_response(text: str) -> str:    
    return 'ok, got it bro'

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    message_type: str = update.message.chat.type
    text: str = update.message.text

    print(f'User {update.message.chat.username} in {message_type}: "{text}"')

    if message_type == 'group' or message_type == 'supergroup':
        response: str = handle_response(text)
    else:
        response: str = handle_response(text) + ' jeje'
    
    #print(f'Bot: {response}')
    await update.message.reply_text(response)

async def error(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print(f'Update {update} caused error {context.error}')


#Codeforces API methods
def get_random_problems():
    # Initialize the API
    cf_api = codeforces_api.CodeforcesApi()

    # Get all problems
    problems = cf_api.problemset_problems()

    print(len(problems))

    # Select 3 random problems
    random_problems = random.sample(problems['problems'], 3)

    # Generate links for the problems
    problem_links = []
    for problem in random_problems:
        link = get_problem_link(problem)
        problem_links.append(link)

    return problem_links


#Others
def read_file(filename, foldername):
    # Get the directory of the current script
    current_dir = os.path.dirname(os.path.realpath(__file__))
    # Construct the full path to the file
    full_path = os.path.join(current_dir, foldername, filename)

    with open(full_path, 'r') as file:
        content = file.read()
    return content

def get_problem_link(problem):
    return f"https://codeforces.com/problemset/problem/{problem.contest_id}/{problem.index}"

def write_dict_to_json(my_dict, foldername, filename):
    # Get the directory of the current script
    current_dir = os.path.dirname(os.path.realpath(__file__))
    # Construct the full path to the file
    full_path = os.path.join(current_dir, foldername, filename)

    # Ensure the database directory exists
    os.makedirs(os.path.dirname(full_path), exist_ok=True)

    with open(full_path, 'w') as file:
        json.dump(my_dict, file)

def read_json_to_dict(filename):
    # Get the directory of the current script
    current_dir = os.path.dirname(os.path.realpath(__file__))
    # Construct the full path to the file
    full_path = os.path.join(current_dir, 'database', filename)

    with open(full_path, 'r') as file:
        my_dict = json.load(file)
    return my_dict


In [ ]:
bot_username = '@cfquestboardbot'
bot_token = read_file('telegram_bot_token', 'tokens')
cf_api_key = read_file('codeforces_api_key', 'tokens')
cf_api_secret = read_file('codeforces_api_secret', 'tokens')


if __name__ == '__main__':
    print('Starting...')


    app = Application.builder().token(bot_token).build()

    #Commands
    app.add_handler(CommandHandler('start', start_command))
    app.add_handler(CommandHandler('help', help_command))
    app.add_handler(CommandHandler('random_problems', random_problems_command))

    #Messages
    app.add_handler(MessageHandler(filters.TEXT, handle_message))

    #Errors
    app.add_error_handler(error)

    #Polls the bot
    print('Polling...')
    await app.initialize() # inits bot, update, persistence
    await app.start()
    await app.updater.start_polling(poll_interval=1)

print('All Good')

In [4]:
#pip install codeforces-api
#update.message.reply_text(text="<a href='https://www.google.com/'>Google</a>", parse_mode=ParseMode.HTML)   to send links